In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.set_printoptions(linewidth=100)
from numba import jit

In [83]:
@jit
def sign(x): 
    if x >= 0:
        return 1
    else:
        return -1

In [84]:
@jit
def qr_householder(A,tol=1e-10):
    
    m,n = A.shape
    V = []
    R = A.copy()
    Q = np.eye(m)
    
    for k in range(n):
        # define the x vector
        x = R[k:,k].copy()
    
        # modify and normalize it 
        v = (sign(x[0])*np.linalg.norm(x)*np.eye(len(x))[0] + x).copy()
        
        v = v / np.linalg.norm(v,2)
        
        # compute transformation to get R
        R[k:,k:] = R[k:,k:] - 2 * np.outer(v, (v@R[k:,k:]))
    
        # Compute Q
        Q[k:,:] = Q[k:,:] - 2 * np.outer(v, (v@Q[k:,:]))
    
    # Fix 0 values and roundoff error
    mask = np.abs(R) < tol
    R[mask] = 0

    return Q.T, R

In [85]:
@jit
def lq_householder(A,tol=1e-10):
    
    m,n = A.shape
    V = []
    L = A.copy()
    Q = np.eye(n)
    
    for k in range(n):

        # do rh reflection
        x = L.T[k:,k].copy()

        # define the vector the the rh reflection
        # modify and normalize it 
        v = (sign(x[0])*np.linalg.norm(x)*np.eye(len(x))[0] + x).copy()
        
        v = v / np.linalg.norm(v,2)
        
        # compute transformation for right hand
        L.T[k:,k:] = L.T[k:,k:] - 2 * np.outer(v, v@L.T[k:,k:])

        # Compute Q^*b for b and element of the identity
        Q[k:,:] = Q[k:,:] - 2 * np.outer(v, (v@Q[k:,:]))
    
    # Fix 0 values and roundoff error
    mask = np.abs(L) < tol
    L[mask] = 0

    return L, Q

In [4]:
def golub_kahan_bidiag(A, tol=1e-10):
    '''
    A function that bidiagonalizes any real matrix A using the Golub Kahan method

    Parameters:
        A (ndarray)(mxn): A real symmetric matrix.
    
    Returns:
        T (ndarray)(mxm): A symmetric tridiagonalization of A
        Q (ndarray)(mxm): An orthogonal matrix where T = Q*AQ
    '''

    def sign(x): return 1 if x >= 0 else -1
    
    # other params
    m,n = A.shape
    T = A.copy()
    U = np.eye(m)
    V = np.eye(n)
    is_upper = np.allclose(A, np.triu(A))


    def lh_reflection(T, U, k):
        # define the x vector
        x = T[k:,k].copy()

        # define the vector the the lh reflection
        # modify and normalize it 
        v = (sign(x[0])*np.linalg.norm(x)*np.eye(len(x))[0] + x).copy()
        
        v = v / np.linalg.norm(v,2)
        
        # compute transformation for left hand
        T[k:,k:] = T[k:,k:] - 2 * np.outer(v, v@T[k:,k:])

        # Compute Q^*b for b and element of the identity
        U[k:,:] = U[k:,:] - 2 * np.outer(v, (v@U[k:,:]))

        return T, U
    
    def rh_reflection(T, V, k):
        # do rh reflection
        x = T.T[k+1:,k].copy()

        # define the vector the the rh reflection
        # modify and normalize it 
        v = (sign(x[0])*np.linalg.norm(x)*np.eye(len(x))[0] + x).copy()
        
        v = v / np.linalg.norm(v,2)
        
        # compute transformation for right hand
        T.T[k+1:,k:] = T.T[k+1:,k:] - 2 * np.outer(v, v@T.T[k+1:,k:])

        # Compute Q^*b for b and element of the identity
        V[k+1:,:] = V[k+1:,:] - 2 * np.outer(v, (v@V[k+1:,:]))

        return T, V
    
    for k in range(n):

        # if the matrix is alread upper triangular then we can
        # skip the lh reflections
        if not is_upper:

            T, U = lh_reflection(T, U, k)

        if k <= n-2:

            T, V = rh_reflection(T, V, k)


    # Fix 0 values and roundoff error
    mask = np.abs(T) < tol
    T[mask] = 0

    return U.T, T, V.T

In [5]:
def lawson_hanson_chan_bidiag(A,tol=1e-10):
    '''
    A function that bidiagonalizes any real matrix A using the Lawson Hanson Chan method

    Parameters:
        A (ndarray)(mxn): A real symmetric matrix.
    
    Returns:
        T (ndarray)(mxm): A symmetric tridiagonalization of A
        Q (ndarray)(mxm): An orthogonal matrix where T = Q*AQ
    '''

    # Compute the QR factorization of A
    Q, R = qr_householder(A, tol)


    # compute the golub-kaham bidiagnolization of R
    U, T, V = golub_kahan_bidiag(R, tol)


    return Q, T, V


In [87]:
L = np.round(np.random.random((6,5))*10, 3)
print(np.linalg.svd(A)[1])

for _ in range(50000):
    Q, R = qr_householder(L)
    L, Q = lq_householder(R)

L

[27.89294394 10.37729574  9.52044449  3.86051673  1.91756588]


array([[30.79084401,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  8.2513359 ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  6.60306146,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -5.60616165,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.99165536],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])